# Approach

1. Load datasets, change multiclass to binary class (positive or negative only), limit to text column
2. Tokenize, stop words, stem
3. TF-IDF
4. Sentiment analysis

In [31]:
import pandas as pd
import torch
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
def data_transformer(df):
    df['Sentiment'] = df['Sentiment'].map({'Positive':'Positive', 'Extremely Positive':'Positive', 
                                           'Negative':'Negative', 'Extremely Negative':'Negative',
                                           'Neutral':'Neutral'
                                          })
    df = df.drop(['UserName','ScreenName','Location','TweetAt'], axis=1)
    return df

In [24]:
df_train = pd.read_csv('./Data/Corona_NLP_train.csv')
df_test = pd.read_csv('./Data/Corona_NLP_test.csv')

In [26]:
df_train = data_transformer(df_train)
df_test = data_transformer(df_test)

In [27]:
df_test

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...
3793,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,Did you panic buy a lot of non-perishable item...,Negative
3795,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,Gov need to do somethings instead of biar je r...,Negative


# Sources:

1. PyTorch TF-IDF:
https://medium.com/swlh/text-classification-using-scikit-learn-pytorch-and-tensorflow-a3350808f9f7


False